## Fake News Classification

Solution: [Universal Sentence Encoder (USE) for English](https://www.aclweb.org/anthology/D18-2029)

This [blog post](https://towardsdatascience.com/using-use-universal-sentence-encoder-to-detect-fake-news-dfc02dc32ae9) reaches ~ 90% accuracy with universal encoder from tf hub. But can we reach the same results with a simple encoder ?

In [7]:
!pip install -U pipenv && pipenv install --dev
!pip install encoder

Installing dependencies from Pipfile.lock (a65489)...
  🐍   ▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉ 0/0 — 00:00:00
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


In [8]:
! pip install fasttext

In [9]:
! pip install contractions

In [10]:
! pip install textacy

In [11]:
! git clone https://github.com/MohamedISSA98/simple-but-tough-to-beat-examples.git

Cloning into 'simple-but-tough-to-beat-examples'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 26 (delta 9), reused 12 (delta 1), pack-reused 0
Unpacking objects: 100% (26/26), done.


In [12]:
import os
os.chdir("/content/simple-but-tough-to-beat-examples")

In [13]:
import encoder

In [14]:
import csv
import itertools
import os

from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tqdm.notebook import tqdm

from encoder import build_from_fasttext_bin
from nn import train_w2v, train_nn, load_model, fasttext
from utils import preprocess_sentence

Download the dataset (from github)

In [15]:
CSV_FILE = 'fake_or_real_news.csv'

! [[ ! -f { CSV_FILE } ]] && wget https://github.com/saadarshad102/Fake-News-Detection-Universal-Sentence-Encoder/raw/master/{ CSV_FILE }
    
def read_news(fname):
    X = []
    y = []
    with open(fname) as f:
        for row_num, row in enumerate(csv.reader(f)):
            if row_num == 0:
                continue
            _, title, text, label = row
            X.append(text)
            y.append(label)
    return X, y

X, y = read_news(CSV_FILE)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

--2021-11-14 20:47:16--  https://github.com/saadarshad102/Fake-News-Detection-Universal-Sentence-Encoder/raw/master/fake_or_real_news.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/saadarshad102/Fake-News-Detection-Universal-Sentence-Encoder/master/fake_or_real_news.csv [following]
--2021-11-14 20:47:17--  https://raw.githubusercontent.com/saadarshad102/Fake-News-Detection-Universal-Sentence-Encoder/master/fake_or_real_news.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30696129 (29M) [text/plain]
Saving to: ‘fake_or_real_news.csv’

fake_or_real_news.c 100%[===================>]  29.27

Build a corpus for word2vec training and pre-process with `textacy` lib:
  - normalize unicode charset.
  - deaccent (rèsume -> resume)
  - unpack contractions (he's --> he is).
  - remove emojis, hashtags, URLs, emails, etc
  - remove punctuation marks
  - strip whitespace
  - lowercase
  
and train word2vec skipgram model as follows;
  - dim = 200
  - lr = relatively low.
  - epochs = 15 (but should probably be ~ 25).
  - ws = 5 (but should probably be ~ 7).
  - sub-word information (minn = 3, maxn = 6).
  
alternatively, we can use a [pre-built model](https://fasttext.cc/docs/en/pretrained-vectors.html).

In [16]:
W2V_PREBUILT_MODEL = 'cc.en.300.bin'
W2V_MODEL = W2V_PREBUILT_MODEL # W2V_PREBUILT_MODEL

if W2V_MODEL == W2V_PREBUILT_MODEL:
    ! [[ ! -f {W2V_PREBUILT_MODEL} ]] && wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/{W2V_PREBUILT_MODEL}.gz
    ! [[ ! -f {W2V_PREBUILT_MODEL} ]] && gzip -d {W2V_PREBUILT_MODEL}.gz
    ! ls -lh {W2V_PREBUILT_MODEL}

model = fasttext.load_model(W2V_MODEL)

--2021-11-14 20:47:30--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G  30.0MB/s    in 2m 24s  

2021-11-14 20:49:54 (29.8 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]

-rw-r--r-- 1 root root 6.8G Jan 18  2019 cc.en.300.bin


word2vec ---> "Simple But Tough to Beat .." encoder

In [17]:
sentence_encoder = build_from_fasttext_bin(model, preprocessor=preprocess_sentence, weighted=True)

del model # free some memory !

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = sentence_encoder.fit_transform(X_train)
X_test = sentence_encoder.transform(X_test)

print('X_train.shape = ', X_train.shape)
print('X_test.shape = ', X_test.shape)

X_train.shape =  (5068, 300)
X_test.shape =  (1267, 300)


Split train/test and transform sentences to their embedding representation

Now we can train a binary classification net:
  - 1 hidden layer (128).
  - dropout ~ [0.2 - 0.5].
  - binary logloss.

In [23]:
MODEL_PT = 'model.h5'

model = train_nn(
    X_train,
    y_train,
    hidden_layers=(128,),
    activation='relu',
    dropout=0.4,
    epochs=25,
    batch_size=32,
    # validation_split=0.1,
    validation_data=(X_test, y_test),
    patience=4,
    shuffle=True,
    optimizer='adam',
    pt=MODEL_PT,
)

Epoch 1/25
159/159 [==============================] - 4s 6ms/step - loss: 0.6286 - accuracy: 0.7399 - val_loss: 0.5318 - val_accuracy: 0.7932
Epoch 2/25
159/159 [==============================] - 1s 4ms/step - loss: 0.4844 - accuracy: 0.7954 - val_loss: 0.4216 - val_accuracy: 0.8279
Epoch 3/25
159/159 [==============================] - 1s 5ms/step - loss: 0.4110 - accuracy: 0.8270 - val_loss: 0.3689 - val_accuracy: 0.8540
Epoch 4/25
159/159 [==============================] - 1s 4ms/step - loss: 0.3672 - accuracy: 0.8491 - val_loss: 0.3387 - val_accuracy: 0.8635
Epoch 5/25
159/159 [==============================] - 1s 5ms/step - loss: 0.3416 - accuracy: 0.8625 - val_loss: 0.3181 - val_accuracy: 0.8706
Epoch 6/25
159/159 [==============================] - 1s 4ms/step - loss: 0.3225 - accuracy: 0.8725 - val_loss: 0.3042 - val_accuracy: 0.8777
Epoch 7/25
159/159 [==============================] - 1s 5ms/step - loss: 0.3072 - accuracy: 0.8814 - val_loss: 0.2945 - val_accuracy: 0.8777
Epoch 

91% accuracy with a pretty simple encoder ! that's nice !

In [24]:
model = load_model(MODEL_PT)
preds = model.predict_classes(X_test, batch_size=32)
preds = preds.reshape(preds.shape[0])

report = classification_report(y_test, preds, target_names=label_encoder.classes_)
print(report)

              precision    recall  f1-score   support

        FAKE       0.90      0.89      0.89       628
        REAL       0.89      0.90      0.90       639

    accuracy                           0.90      1267
   macro avg       0.90      0.90      0.90      1267
weighted avg       0.90      0.90      0.90      1267



/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:454: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
